In [ ]:
#!pip install awswrangler

In [2]:
import awswrangler as wr
import pandas as pd

In [3]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name= 'analytics'
table_name = 'daily_check'

In [5]:
import sagemaker
sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


'arn:aws:iam::283731589572:role/service-role/SageMaker-ExecutionRole-20240102T104128'

In [4]:
# Read data from Athena
df_daily_check = wr.athena.read_sql_query(
    f'SELECT * FROM {origin_name}.{database_name}.{table_name};',
    database=database_name
)


ClientError: An error occurred (AccessDeniedException) when calling the GetWorkGroup operation: You are not authorized to perform: athena:GetWorkGroup on the resource. After your AWS administrator or you have updated your permissions, please try again.

In [ ]:
df_daily_check = pd.read_excel("daily_check.xlsx")

In [ ]:
# Convertir todos los nombres de columna a minúsculas
df_daily_check.columns = df_daily_check.columns.str.lower()

In [ ]:
df_test = pd.read_csv("df_train_prueba2.csv")

In [ ]:
df_test

In [ ]:
# Me traigo solamente las columnas de payer_country y la del encoder para poder cruzar los datos
df_translate = pd.read_csv("df_final_21_10_2023.csv")[['payer_country','payer_country_encoder']]

In [ ]:
# Me quedo con una tabla simple, con ocurrencias unicas
df_translate = df_translate.drop_duplicates(subset='payer_country_encoder')

In [ ]:
df_translate.to_excel('translate.xlsx')

In [ ]:
# Al df_test le agrego la correspondencia payer_country_encoder <> payer_country
df_final = pd.merge(df_test, df_translate, on=['payer_country_encoder'])

In [ ]:
#Separo el nombre del pais
df_final['country'] = df_final['payer_country'].str.split('_').str[-1]

In [ ]:
df_final['date'] = pd.to_datetime(df_final['date'])

In [ ]:
df_daily_check['payer_country'] = df_daily_check['PAYER']+'_'+df_daily_check['COUNTRY']

In [ ]:
#Traigo 'amount' de la tabla de transacciones diarias 'daily_check'
df_test = pd.merge(df_final, df_daily_check, on=['date','payer_country', 'country'], how='left')

In [ ]:
df_test.drop(['payer','tx','promedio x tx'], axis=1, inplace=True)

In [ ]:
#Elimino registros donde no tengo amount
df_test = df_test[~df_test['amount'].isnull()]

### MAPE @ MEAN

In [ ]:
statistic = 'mean' # Seleccionar 'mean' o 'p50'

In [ ]:
#Calculo el valor absoluto de la diferencia entre 'amount' y 'mean'
df_test['abs_error'] = abs(df_test['amount'] - df_test[statistic])

In [ ]:
#Calculo el % de error absoluto para cada fila
df_test['ape'] = df_test['abs_error'] / df_test['amount']

In [ ]:
df_test

In [ ]:
#Calculo el MAPE por payer_country
mape_payer_country = df_test.groupby('payer_country')['ape'].mean()
mape_payer_country

In [ ]:
#Calculo el MAPE por country
mape_country = df_test.groupby('country')['ape'].mean()
mape_country